# Programming with Python

## Lecture 14: Web APIs

### Armen Gabrielyan

#### Yerevan State University / ASDS

#### 24 May, 2025

# Web serving

## HTTP (HyperText Transfer Protocol)

HTTP is the foundation of data communication on the World Wide Web. It's an application layer protocol that enables the transfer of hypermedia documents like HTML.

### Request-Response Model
- **Client** (usually a browser) sends a request to a server
- **Server** processes the request and returns a response

### HTTP Methods (Verbs)
- **GET**: Retrieve data (most common)
- **POST**: Submit data (forms, file uploads)
- **PUT**: Update existing resources
- **DELETE**: Remove resources
- **HEAD**: Get headers only (no body)
- **OPTIONS**: Check available methods/options
- **PATCH**: Partial resource modification

### Status Codes
- **1xx**: Informational responses
- **2xx**: Success (200 OK is most common)
- **3xx**: Redirection
- **4xx**: Client errors (404 Not Found, 403 Forbidden)
- **5xx**: Server errors

### Headers
Provide metadata about the request or response:
- Content-Type
- Authorization
- User-Agent
- Cache-Control
- Cookie


### HTTPS
HTTP with encryption using TLS/SSL, providing:
- Authentication
- Data integrity
- Confidentiality

### Basic HTTP Request Example
```
GET /index.html HTTP/1.1
Host: www.example.com
User-Agent: Mozilla/5.0
Accept: text/html
```

### Basic HTTP Response Example
```
HTTP/1.1 200 OK
Date: Sat, 17 May 2025 10:30:45 GMT
Content-Type: text/html
Content-Length: 1234

<!DOCTYPE html>
<html>
...
```

## IP (Internet Protocol)

**IP** stands for **Internet Protocol**, and an **IP address** is a unique identifier assigned to each device connected to a network — like your computer, phone, or web server.

The following are types of IP:

### 1. **IPv4** (most common)

* Format: `x.x.x.x` (four numbers from 0–255)
* Example: `192.168.1.1`
* 32-bit address (about 4.3 billion possible addresses)

### 2. **IPv6** (newer, supports more devices)

* Format: eight groups of hexadecimal numbers
* Example: `2001:0db8:85a3:0000:0000:8a2e:0370:7334`
* 128-bit address (significantly more space)

### IP in Web

When you visit a website:

1. Your browser looks up the website’s **domain name** (e.g. `google.com`) via **DNS**.
2. DNS returns the **server's IP address**.
3. Your browser connects to that **IP** to send an HTTP/HTTPS request.


## Host

A host refers to any device connected to a network that can send or receive data. In web serving:

- **Hostname**: A human-readable name that identifies a host on a network (e.g., `www.example.com`)
- **IP Address**: The numerical identifier assigned to each device (e.g., `192.168.1.1` or `2001:db8::1`)
- **localhost**: Special hostname that refers to the current machine (`127.0.0.1` in IPv4)

Hosts can be servers providing services or clients consuming them.

## Ports

A port is a virtual point where network connections start and end. Ports are identified by numbers (0-65535):

- **Well-known ports** (0-1023): Reserved for common services
  - HTTP: 80
  - HTTPS: 443
  - FTP: 21
  - SSH: 22
  - SMTP: 25

When setting up a web server, you configure it to listen on specific host(s) and port(s). For example:

- `0.0.0.0:80` means "listen on port 80 across all network interfaces"
- `localhost:8080` means "listen only on port 8080 for connections from the local machine"

The combination of host and port creates a socket, which applications use to communicate over networks.

## Web servers

A web server is software and hardware that accepts requests via HTTP/HTTPS and serves web content to clients (typically browsers). Some well known web servers are Apache HTTP Server and Nginx.

## WSGI

**WSGI** stands for [**Web Server Gateway Interface**](https://wsgi.readthedocs.io/en/latest/what.html). It's a specification that defines how web servers communicate with Python web applications. WSGI is a **standard interface** between web server software and Python applications or frameworks, such as Flask or Django.

WSGI provides a **common API** so you can plug in any compliant server (like Gunicorn, uWSGI) and any compliant framework.

#### It works as follows:

1. A **web server** receives a request.
2. The server **calls the Python application** using the WSGI interface.
3. The application returns a **response**, which the server sends back to the client.

#### Popular WSGI servers include:

* [**Gunicorn**](https://gunicorn.org/)
* [**uWSGI**](https://uwsgi-docs.readthedocs.io/en/latest/)
* [**mod\_wsgi**](https://modwsgi.readthedocs.io/en/master/)

#### Popular WSGI frameworks include:

* [**Django**](https://www.djangoproject.com/)
* [**Flask**](https://flask.palletsprojects.com/en/stable/)

### WSGI app example

```python
# main.py

def app(environ, start_response):
    status = "200 OK"
    headers = [("Content-type", "text/plain; charset=utf-8")]
    start_response(status, headers)
    return [b"Hello, World!"]
```

* `environ`: A *dictionary* containing **request data**.
* `start_response`: A *function* used to **send HTTP status and headers**.

This app can be run using a WSGI server like `gunicorn`:

```bash
gunicorn main:app
```

`gunicorn` can be installed as follows:

In [ ]:
! pip install gunicorn

**See practical example 1.**

## ASGI

**ASGI** stands for [**Asynchronous Server Gateway Interface**](https://asgi.readthedocs.io/en/latest/). It's a specification that allows Python web applications and frameworks to handle asynchronous, event-driven communication — like WebSockets and long-lived HTTP connections — in addition to standard HTTP.


Before ASGI, Python web apps primarily used **WSGI**, which is synchronous and blocks I/O. ASGI is the successor designed to support:

* Asynchronous request handling using `async/await`
* WebSockets
* Background tasks
* Long-lived connections (like Server-Sent Events)


#### Popular ASGI servers include:

* [**Uvicorn**](https://www.uvicorn.org/)
* [**Hypercorn**](https://github.com/pgjones/hypercorn)

#### Popular ASGI frameworks include:

* [**Starlette**](https://www.starlette.io/)
* [**FastAPI**](https://fastapi.tiangolo.com/)
* [**Falcon**](https://falconframework.org/)
* [**Quart**](https://github.com/pallets/quart)

### ASGI app example

```python
async def app(scope, receive, send):
    assert scope["type"] == "http"

    await receive()  # Wait for the request

    await send(
        {
            "type": "http.response.start",
            "status": 200,
            "headers": [[b"content-type", b"text/plain"]],
        }
    )

    await send(
        {
            "type": "http.response.body",
            "body": b"Hello, ASGI!",
        }
    )

```

* `scope`: A *dictionary* containing connection metadata (like HTTP method, path, headers, client IP, etc.) — it defines the context of the incoming request or connection.

* `receive`: an *async function* your app calls to **receive incoming events**, such as HTTP request bodies or WebSocket messages.

* `send`: an *async function* your app uses to **send events/responses** back to the client, such as HTTP response headers and body, or WebSocket messages.

This app can be run using a ASGI server like `uvicorn`:

```bash
uvicorn main:app
```

`uvicorn` can be installed as follows:

In [ ]:
! pip install uvicorn

**See practical example 2.**

## Pydantic

[**Pydantic**](https://docs.pydantic.dev/latest/) is a Python library that uses type hints to validate data and parse it into structured objects. It's incredibly useful for APIs, configuration management and data validation.

In [ ]:
! pip install pydantic

### Basic usage

We can create a Pydantic model by inheriting from `BaseModel` class.

In [ ]:
from pydantic import BaseModel


class User(BaseModel):
    name: str
    age: int
    email: str
    is_active: bool = True # default value
    bio: str | None = None # optional field

In [ ]:
user = User(name="Alice", age=30, email="alice@example.com")
user

In [ ]:
# convert to a dictionary
user.model_dump()

### Automatic validation

Pydantic automatically validates data types and raises errors for invalid data.

In [ ]:
invalid_user = User(name="Bob", age="not a number", email="bob@example.com")

### Field validation

In [ ]:
from pydantic import BaseModel, Field, field_validator


class Product(BaseModel):
    name: str = Field(..., max_length=100)
    price: float = Field(..., gt=0) # greater than 0
    quantity: int = Field(default=0, ge=0) # greater than or equal to 0
    
    @field_validator("name")
    def name_must_not_be_empty(cls, value: str) -> str:
        if not value.strip():
            raise ValueError("Name cannot be empty")
        return value.title() # capitalize

In [ ]:
product = Product(name="apple", price=1.5, quantity=10)
product

In [ ]:
invalid_product = Product(name="", price=1.5, quantity=10)

### Nested models

In [ ]:
class Address(BaseModel):
    street: str
    city: str
    country: str

class Company(BaseModel):
    name: str
    address: Address
    employees: list[User]

In [ ]:
company = Company(
    name="Tech Corp",
    address={
        "street": "123 Main St",
        "city": "New York",
        "country": "USA"
    },
    employees=[
        {"name": "Alice", "age": 30, "email": "alice@example.com"},
        {"name": "Bob", "age": 25, "email": "bob@example.com"}
    ]
)
company

### JSON parsing

In [ ]:
# parse from JSON string
json_data = '{"name": "Charlie", "age": 28, "email": "charlie@example.com"}'
user = User.model_validate_json(json_data)
user

In [ ]:
# parse from dictionary
data_dict = {"name": "David", "age": 35, "email": "david@example.com"}
user = User.model_validate(data_dict)
user

In [ ]:
# convert to JSON string
user.model_dump_json()

Learn more about Pydantic features at its [documentation](https://docs.pydantic.dev/latest/).

## FastAPI

[**FastAPI**](https://fastapi.tiangolo.com/) is a modern, high-performance Python web framework for building APIs. It's built on standard Python type hints and provides automatic API documentation, data validation and serialization.

### Installation

In [ ]:
! pip install "fastapi[standard]"

### Basic application

```python
# main.py

from fastapi import FastAPI

app = FastAPI()

@app.get("/")
def read_root():
    return {"Hello": "World"}

@app.get("/items/{item_id}")
def read_item(item_id: int, q: str = None):
    return {"item_id": item_id, "q": q}
```

This app can be run using a ASGI server like `uvicorn`:

```bash
uvicorn main:app --reload
```

**See practical example 3.**

Learn more about FastAPI features at its [documentation](https://fastapi.tiangolo.com/).

## SQLModel

[**SQLModel**](https://sqlmodel.tiangolo.com/) is a library that combines SQLAlchemy and Pydantic, allowing you to define database models that work seamlessly with both database operations and API serialization. It is based on type annotations.

### Installation

In [ ]:
! pip install sqlmodel

### Basic usage

We can define a table schema by inheriting from `SQLModel` class. `Field` type can be used to define columns and their attributes.

In [ ]:
from enum import StrEnum
from datetime import datetime

from sqlmodel import SQLModel, Field


class Status(StrEnum):
    TODO = "todo"
    IN_PROGRESS = "in_progress"
    COMPLETED = "completed"

class Priority(StrEnum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"

class Task(SQLModel, table=True):
    id: int | None = Field(default=None, primary_key=True)
    title: str = Field(max_length=200)
    description: str | None = None
    status: Status = Field(default=Status.TODO)
    priority: Priority = Field(default=Priority.MEDIUM)
    created_at: datetime = Field(default_factory=datetime.now)
    due_date: datetime | None = None
    completed_at: datetime | None = None

### Database setup

In [ ]:
from sqlmodel import create_engine, Session


# Create database engine
engine = create_engine("sqlite:///tasks.db")

# Create all tables
SQLModel.metadata.create_all(engine)

# Create a session
with Session(engine) as session:
    pass

### CRUD

#### Create records

In [ ]:
# Create a single task

task = Task(
    title="Learn SQLModel",
    description="Learn SQLModel by following examples",
    status=Status.IN_PROGRESS,
    priority=Priority.HIGH,
    due_date=datetime(2025, 5, 31)
)

print("Task ID", task.id)

with Session(engine) as session:
    session.add(task)
    session.commit()
    session.refresh(task) # Get the generated ID

print("Task ID", task.id)

In [ ]:
# Create multiple tasks

tasks = [
    Task(title="Buy groceries", priority=Priority.LOW),
    Task(title="Finish project", priority=Priority.HIGH, description="Complete university project"),
    Task(title="Call my parents", priority=Priority.MEDIUM)
]

with Session(engine) as session:
    for task in tasks:
        session.add(task)
    session.commit()

#### Read records

In [ ]:
from sqlmodel import select


def get_all_tasks() -> list[Task]:
    with Session(engine) as session:
        statement = select(Task)
        tasks = session.exec(statement).all()
        return tasks

def get_task_by_id(task_id: int) -> Task | None:
    with Session(engine) as session:
        task = session.get(Task, task_id)
        return task

def get_tasks_by_status(status: Status) -> list[Task]:
    with Session(engine) as session:
        statement = select(Task).where(Task.status == status)
        tasks = session.exec(statement).all()
        return tasks

def get_tasks_by_priority(priority: Priority) -> list[Task]:
    with Session(engine) as session:
        statement = select(Task).where(Task.priority == priority)
        tasks = session.exec(statement).all()
        return tasks

In [ ]:
# Get all tasks

get_all_tasks()

In [ ]:
# Get task by id = 2

get_task_by_id(2)

In [ ]:
# Get task by id = 10 (not found)

get_task_by_id(10) is None

In [ ]:
# Get all tasks that are in todo status

get_tasks_by_status(Status.TODO)

In [ ]:
# Get all high priority tasks

get_tasks_by_priority(Priority.HIGH)

#### Updating records

In [ ]:
def complete_task(task_id: int) -> Task | None:
    with Session(engine) as session:
        task = session.get(Task, task_id)
        
        if task:
            task.status = Status.COMPLETED
            task.completed_at = datetime.now()
            
            session.add(task)
            session.commit()
            session.refresh(task)
            
            return task

def update_task_priority(task_id: int, new_priority: Priority) -> Task | None:
    with Session(engine) as session:
        task = session.get(Task, task_id)
        
        if task:
            task.priority = new_priority
            
            session.add(task)
            session.commit()
            session.refresh(task)
            
            return task

In [ ]:
# Complete task by id = 2

complete_task(2)

In [ ]:
# Get task by id = 2

get_task_by_id(2)

In [ ]:
# Set the priority of task by id = 4 to high

update_task_priority(4, Priority.HIGH)

In [ ]:
# Get task by id = 4

get_task_by_id(4)

#### Delete records

In [ ]:
def delete_task(task_id: int) -> bool:
    with Session(engine) as session:
        task = session.get(Task, task_id)
        
        if task:
            session.delete(task)
            session.commit()
            
            return True
        
        return False

def delete_completed_tasks() -> int:
    with Session(engine) as session:
        statement = select(Task).where(Task.status == Status.COMPLETED)
        completed_tasks = session.exec(statement).all()
        
        for task in completed_tasks:
            session.delete(task)
        
        session.commit()

        return len(completed_tasks)

In [ ]:
# Delete completed tasks

delete_completed_tasks()

In [ ]:
# Get all tasks

get_all_tasks()

In [ ]:
# Delete task by id = 1

delete_task(1)

In [ ]:
# Get all tasks

get_all_tasks()

Learn more about SQLModel features at its [documentation](https://sqlmodel.tiangolo.com/).

## `uv` package manager

[**uv**](https://docs.astral.sh/uv/) is an extremely fast Python package and project manager written in Rust. It's designed as a drop-in replacement for `pip`, `pip-tools` and `virtualenv` with significantly better performance.

### Install packages via `pip`

```bash
# Install packages like pip
uv pip install httpx fastapi

# Install from requirements.txt
uv pip install -r requirements.txt

# Install specific versions
uv pip install "pydantic>=2.11.0,<3.0.0"

# Install with extras
uv pip install "fastapi[standard]"
```

### Virtual environment

```bash
# Create virtual environment
uv venv

# Create with specific Python version
uv venv --python 3.12

# Create in specific directory
uv venv myproject-env

# Activate (same as regular venv)

# Linux/Mac:
source .venv/bin/activate

# Windows:
.venv\Scripts\activate
```

### Create new projects

```bash
# Create a new Python project
uv init my-project

cd my-project

# Project structure created:

# my-project/
# ├── pyproject.toml
# ├── README.md
# ├── main.py
# └── .python-version
```

### Add dependencies

```bash
# Add a dependency to your project
uv add httpx

# Add development dependencies
uv add --dev pytest ruff

# Add with version constraints
uv add "fastapi>=0.115.0"
```

### Manage dependencies

```bash
# Remove a dependency
uv remove requests

# Install project dependencies
uv sync

# Install only production dependencies (no dev dependency)
uv sync --no-dev
```

### Execute Python code


```bash
# Run Python with uv (automatically manages environment)
uv run python main.py

# Run with specific Python version
uv run --python 3.12 python script.py

# Run a package/module
uv run -m pytest

# Run inline Python
uv run python -c "import sys; print(sys.version)"
```

Learn more about `uv` features at its [documentation](https://docs.astral.sh/uv/).

## Example FastAPI application

See `books-api` folder for a FastAPI application with database functionality using `uv` as our package manager.

The following are steps to reproduce the application.

1. Create the project

```bash
# Create a new project
uv init book-api

cd book-api

# The initial structure is as follows:

# book-api/
# ├── pyproject.toml
# ├── README.md
# ├── src/
# │   └── book_api/
# │       └── __init__.py
# └── .python-version
```

2. Add dependencies

```bash
# Add fastapi and related packages
uv add "fastapi[standard]"

# Add sqlmodel
uv add sqlmodel
```

3. Project structure

Create the following directory structure:

```bash
mkdir -p src/book_api/{models,routers,database,services}
touch src/book_api/{main,database/__init__,models/__init__,routers/__init__,services/__init__}.py
touch src/book_api/database/connection.py
touch src/book_api/models/book.py
touch src/book_api/routers/books.py
touch src/book_api/services/book_service.py

# Final structure:

# ├── pyproject.toml
# ├── README.md
# ├── src
# │   └── book_api
# │       ├── __init__.py
# │       ├── database
# │       │   ├── __init__.py
# │       │   └── connection.py
# │       ├── main.py
# │       ├── models
# │       │   ├── __init__.py
# │       │   └── book.py
# │       ├── routers
# │       │   ├── __init__.py
# │       │   └── books.py
# │       └── services
# │           ├── __init__.py
# │           └── book_service.py
```

4. Add database models to `src/book_api/models/book.py`

5. Add database connection to `src/book_api/database/connection.py`

6. Add CRUD operations to `src/book_api/services/book_service.py`

7. Add API routes to `src/book_api/routers/books.py`

8. Add main application code to `src/book_api/main.py`.

9. Run development server

```bash
uv run uvicorn src.book_api.main:app --reload
```

Thank you for getting so far in this course 🎉